# LOADING META-DATA IN CSV FILE

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File ‘00-start-eva-server.ipynb’ already there; not retrieving.



nohup eva_server >> eva.log 2>&1 &
('03-18-2023 23:59:49 WARNING[create_udf_executor:create_udf_executor.py:exec:0039] UDF FastRCNNObjectDetector already exists, nothing added.\n', '03-18-2023 23:59:49 WARNING[create_udf_executor:create_udf_executor.py:exec:0039] UDF ArrayCount already exists, nothing added.\n', '03-18-2023 23:59:49 WARNING[create_udf_executor:create_udf_executor.py:exec:0039] UDF Timestamp already exists, nothing added.\n')
('03-18-2023 23:59:49 WARNING[create_udf_executor:create_udf_executor.py:exec:0039] UDF Crop already exists, nothing added.\n', '03-18-2023 23:59:49 WARNING[create_udf_executor:create_udf_executor.py:exec:0039] UDF YoloV5 already exists, nothing added.\n', '03-18-2023 23:59:49 WARNING[create_udf_executor:create_udf_executor.py:exec:0039] UDF Open already exists, nothing added.\n')
('03-18-2023 23:59:49 WARNING[create_udf_executor:create_udf_executor.py:exec:0039] UDF Similarity already exists, nothing added.\n', '03-18-2023 23:59:49 WARNING[create_

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

cursor = connection.cursor()

### Video Files
 - Download a subset of the Berkeley Deepdrive dataset from Dropbox

In [3]:
# sample dataset of 2 videos
!wget -nc https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip

# unzip
!unzip -n bddtest.zip

File ‘bddtest.zip’ already there; not retrieving.



Archive:  bddtest.zip


## Load 2 BDD videos

In [4]:
cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load VIDEO failed: encountered unexpected error Duplicate File: bddtest/videos/bddtest_00a2e3ca5c856cde.mp4 already exists in the table bddtest_1
@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load VIDEO failed: encountered unexpected error Duplicate File: bddtest/videos/bddtest_00a395fed60c0b47.mp4 already exists in the table bddtest_2


## Create a BDD Meta-data Table

In [5]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.011930491775274277


## Load the 2 corresponding CSV files associated with the 2 previously loaded videos

In [6]:
cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                          CSV  Number of loaded frames
0  bddtest/info/bddtest_00a2e3ca5c856cde.csv                     3401
@query_time: 0.3905692268162966


@status: ResponseStatus.SUCCESS
@batch: 
                                          CSV  Number of loaded frames
0  bddtest/info/bddtest_00a395fed60c0b47.csv                     4355
@query_time: 0.4162788428366184


## Run a Query over the Video dataset


In [7]:
cursor.execute("""SELECT id, FastRCNNObjectDetector(data) 
                  FROM bddtest_1 
                  WHERE id < 20;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
     bddtest_1.id  \
0              0   
1              1   
2              2   
3              3   
4              4   
5              5   
6              6   
7              7   
8              8   
9              9   
10            10   
11            11   
12            12   
13            13   
14            14   
15            15   
16            16   
17            17   
18            18   
19            19   

                                                                          fastrcnnobjectdetector.labels  \
0   [stop sign, car, stop sign, car, car, car, car, car, car, car, car, stop sign, car, car, car, ca...   
1   [stop sign, car, car, stop sign, car, car, car, car, car, car, car, car, stop sign, stop sign, c...   
2    [stop sign, car, car, stop sign, car, car, car, car, car, car, car, stop sign, car, car, car, car]   
3    [stop sign, car, car, stop sign, car, car, car, car, car, car, car, stop sign, car, car, car, car]   
4 

## Run a query over the Meta-data Table

In [8]:
cursor.execute("""SELECT *
                  FROM bddtestmeta
                  WHERE bddtestmeta.video_id = 4;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
        bddtestmeta.id  bddtestmeta.frame_id  bddtestmeta.video_id  \
0                8185                     0                     4   
1                8186                     0                     4   
2                8187                     0                     4   
3                8188                     0                     4   
4                8189                     0                     4   
...               ...                   ...                   ...   
10198           11581                  1197                     4   
10199           11582                  1197                     4   
10200           11583                  1197                     4   
10201           11584                  1197                     4   
10202           11585                  1197                     4   

      bddtestmeta.dataset_name bddtestmeta.label  \
0                      bddtest               car   
1                      bd